# Development script for Lambert problem solver via Universal Variables

Formulation follows Curtis Chapter 5.3 Lambert's Problem pg.247-

Originally derived in Bate, Mueller, White (1971) Fundamentals of Astronautics

Python implementation by Yuri Shimane

In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from numpy import linalg as linalg
from numpy.linalg import norm

In [2]:
# define functions that will be used repeatedly in iterative solving step
def _Stumpff_S(z):
    """
    Stumpff function S(z)
    Args:
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of Stumpff functio S(z) evaluated for input z
    """
    if z > 0:
        S = (np.sqrt(z) - np.sin(np.sqrt(z)))/np.power(z,1.5)
    elif z == 0:
        S = 1/6
    elif z < 0:
        S = (np.sinh(np.sqrt(-z)) - np.sqrt(-z))/np.power(-z,1.5)

    return S


def _Stumpff_C(z):
    """
    Stumpff function C(z)
    Args:
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of Stumpff functio S(z) evaluated for input z
    """
    if z > 0:
        C = (1 - np.cos(np.sqrt(z)))/z
    elif z == 0:
        C = 1/2
    elif z < 0:
        C = (np.cosh(np.sqrt(-z)) - 1)/(-z)

    return C

def _y_538(r1,r2,A,z):
    """
    Intermediate function in Lambert problem derivation (eq.5.38 in Curtis)
    Args:
        r1 (1x3 numpy array): position vector of departure
        r2 (1x3 numpy array): position vector of arrival
        A (float): intermediate value related only to input parameters
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of function evaluated for input z
    """

    y = norm(r1) + norm(r2) + A*(z*_Stumpff_S(z) - 1)/np.sqrt(_Stumpff_C(z))

    return y

In [12]:
# Lambert solver
def lambert(r1, r2, tof, mu, grade='pro', method=None, **kwargs):
    """
    Function takes in classic parameters to Lambert problem to determine orbitalelements
    Args:
        r1 (1x3 numpy array): initial position vector of departure [km]
        r2 (1x3 numpy array): final position vector of arrival [km]
        tof (float): time of flight [s]
        mu (float): gravitational parameter [km^3/s^2]
        grade (str): trajectory orientation ('pro' for prograde or 'retro' for retrograde)
        **kwargs: other arguments for scipy.optimize.root_scalar
    Returns:
        (tuple): velocity vector at position 1 and 2 of solution trajectory to Lambert problem
    """
    # boundary values of problem
    print('========================= LAMBERT\'S PROBLEM =========================')
    print(f'Transfering from r1: {r1} [km] \n              to r2: {r2} [km] \nin time of flight: {tof/(60*60*24)} [days]')
    print('=====================================================================')

    # compute dtheta [rad]
    dtheta = np.arccos( np.dot(r1,r2)/(norm(r1)*norm(r2)) )
    c12 = np.cross(r1,r2)
    
    # update dtheta for special cases
    if grade=='retro':
        if c12[2] >= 0:
            dtheta = 2*np.pi - dtheta
    else:
        if c12[2] <= 0:
            dtheta = 2*np.pi - dtheta
            
    print('dtheta: {}'.format(dtheta*180/(np.pi)))
    
    # compute input parameter A where A = sin(dtheta) * sqrt[r1*r2 / (1 - cos(dtheta))]
    A = np.sin(dtheta) * np.sqrt(norm(r1)*norm(r2)/(1 - np.cos(dtheta)))
    
    # if A = 0, switch to Hohmann Transfer? - FIXME
    if A == 0.0:
        raise RuntimeError(f"Cannot compute orbit, \Delta\Theta is {dtheta}")
    print(f'Value of A: {A}')
    
    # Scipy - NR method scipy.optimize.rootscalar
    def residue_Fz(z,r1,r2,A):
        """Function computes residue of F(z) as defined by Curtis eq. (5.40)
        Args:
            z (float): value of z at which function F is evaluated
            r1 (1x3 numpy array): initial position vector of departure [km]
            r2 (1x3 numpy array): final position vector of arrival [km]
            A (float): variable A predefined in derivation of solution to Lambert's problem
        Returns:
            (tuple): tuple containing residue of F computed at z and Fdot eq. (5.43)
        """
        residue = np.power(_y_538(r1,r2,A,z)/_Stumpff_C(z), 3/2) * _Stumpff_S(z) + A*np.sqrt(_y_538(r1,r2,A,z)) - np.sqrt(mu)*tof
        
        if z == 0:
            Fdot = np.sqrt(2) * np.power(_y_538(r1,r2,A,0),1.5)/40 + (A/8)*(np.sqrt(_y_538(r1,r2,A,0)) + A*np.sqrt(1/(2*_y_538(r1,r2,A,0))))
        else:
            Fdot = np.power(_y_538(r1,r2,A,z)/_Stumpff_C(z), 1.5) * (((1/(2*z)) * (_Stumpff_C(z) - 3*_Stumpff_S(z)/(2*_Stumpff_C(z)))) + 3*np.power(_Stumpff_S(z),2)/(4*_Stumpff_C(z))) + (A/8)*(3*_Stumpff_S(z)*np.sqrt(_y_538(r1,r2,A,z))/_Stumpff_C(z) + A*np.sqrt(_Stumpff_C(z)/_y_538(r1,r2,A,z)))
    
        return residue, Fdot
    
    # Scipy - prepare initial conditions to solve iteratively
    # FIXME - if orbit is retrograde, override bracket_window to only have positive z-values
    #bracket_window = (0.1,5000)
    # FIXME - currently, most robust way is to have a relatively good initial guess...
    z0 = -10
    F0, Fdot0 = residue_Fz(z0,r1,r2,A)
    print(f'Initially pre-selected z0: {z0}, where F0 = {F0}')
    # if value of F is too big and is NaN, reduce z0 until it isn't
    if np.isnan(F0) == True:
        while np.isnan(F0) == True:
            z0 = z0*0.5
            F0, Fdot0 = residue_Fz(z0,r1,r2,A)
            print(f'Updated z0: {z0}, where F0 = {F0}')
    
    if F0 < 0:
        while F0 < 0:
            z0 = z0 + 1
            F0, Fdot0 = residue_Fz(z0,r1,r2,A)
    
    print(f'Scipy will use {z0} as z0 initial guess')
    
    # Scipy - solve to find z-value
    #sol = opt.root_scalar(residue_Fz, args=(r1,r2,A), fprime=True, bracket=bracket_window, method=method, **kwargs)
    sol = opt.root_scalar(residue_Fz, args=(r1,r2,A), fprime=True, x0=z0, method=method, **kwargs)
    
    if sol.converged:
        z1 = sol.root
    else:
        raise RuntimeError(f'F(z) = 0 calculation failed with initial guess of z {z0}')  # FIXME - document failure
    

    # display orbit type
    if z1 > 0:
        print(f'Transfer trajectory is an ellipse; z = {z1}')
    elif z1 == 0:
        print(f'Transfer trajectory is a parabola; z = {z1}')
    elif z1 < 0:
        print(f'Transfer trajectory is a hyperbolla; z = {z1}')

    # calculate Lagrange functions
    f    = 1 - _y_538(r1,r2,A,z1)/norm(r1)
    g    = A*np.sqrt(_y_538(r1,r2,A,z1)/mu)
    gdot = 1 - _y_538(r1,r2,A,z1)/norm(r2)
    print(f'Lagrange functions f: {f}, g: {g}, gdot: {gdot}')

    # calculate initial and final velocity vectors
    v1 = (1/g)*(r2 - f*r1)
    v2 = (1/g)*(gdot*r2 - r1)
    print('=========================== SOLUTION ===========================')
    print(f'Velocity at r1: {v1} [km/s]\nvelocity at r2: {v2} [km/s]')
    print('================================================================')
    return v1, v2


In [13]:
# initial and final position vectors [km]
r_in = np.array([5000, 10000, 2100])
r_fn = np.array([-14600, 2500, 7000])

# time of flight [s]
dt = 4800

# gravitational parameter [km^3/s^2]
mu_E = 398600

# solve Lamert problem in prograde
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt, mu=mu_E, grade='pro')

========================= LAMBERT'S PROBLEM =========================
Transfering from r1: [ 5000 10000  2100] [km] 
              to r2: [-14600   2500   7000] [km] 
in time of flight: 0.05555555555555555 [days]
dtheta: 100.29252420729621
Value of A: 12372.272033956451
Initially pre-selected z0: -10, where F0 = nan
Updated z0: -5.0, where F0 = nan
Updated z0: -2.5, where F0 = -2081391.8299109235
Scipy will use 3.5 as z0 initial guess
Transfer trajectory is an ellipse; z = 3.4489639564339423
Lagrange functions f: -0.5188925942155815, g: 2575.9433324300862, gdot: -0.05465796566093717
=========================== SOLUTION ===========================
Velocity at r1: [-4.66063709  2.98489716  3.14047066] [km/s]
velocity at r2: [-1.63124462 -3.93511953 -0.96376567] [km/s]


C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in sqrt


In [14]:
# solve Lamert problem in prograde
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt, mu=mu_E, grade='pro')

========================= LAMBERT'S PROBLEM =========================
Transfering from r1: [ 5000 10000  2100] [km] 
              to r2: [-14600   2500   7000] [km] 
in time of flight: 0.05555555555555555 [days]
dtheta: 100.29252420729621
Value of A: 12372.272033956451
Initially pre-selected z0: -10, where F0 = nan
Updated z0: -5.0, where F0 = nan
Updated z0: -2.5, where F0 = -2081391.8299109235
Scipy will use 3.5 as z0 initial guess
Transfer trajectory is an ellipse; z = 3.4489639564339423
Lagrange functions f: -0.5188925942155815, g: 2575.9433324300862, gdot: -0.05465796566093717
=========================== SOLUTION ===========================
Velocity at r1: [-4.66063709  2.98489716  3.14047066] [km/s]
velocity at r2: [-1.63124462 -3.93511953 -0.96376567] [km/s]


C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in sqrt


In [15]:
# solve Lambert problem in retrograde
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt, mu=mu_E, grade='retro')

========================= LAMBERT'S PROBLEM =========================
Transfering from r1: [ 5000 10000  2100] [km] 
              to r2: [-14600   2500   7000] [km] 
in time of flight: 0.05555555555555555 [days]
dtheta: 259.70747579270375
Value of A: -12372.272033956451
Initially pre-selected z0: -10, where F0 = -1669046.4526966834
Scipy will use 10 as z0 initial guess
Transfer trajectory is an ellipse; z = 9.622734027582846
Lagrange functions f: -1.470581086774787, g: -3285.2795382038316, gdot: -0.7154722018537014
=========================== SOLUTION ===========================
Velocity at r1: [ 2.20592935 -5.2372441  -3.07073421] [km/s]
velocity at r2: [-1.65766538  3.58833407  2.16368359] [km/s]


In [7]:
# solve Lambert problem in prograde with more extreme tof
dt2 = 2000
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt2, mu=mu_E, grade='pro')

========================= LAMBERT'S PROBLEM =========================
Transfering from r1: [ 5000 10000  2100] [km] 
              to r2: [-14600   2500   7000] [km] 
in time of flight: 0.023148148148148147 [days]
dtheta: 100.29252420729621
Value of A: 12372.272033956451
Initially pre-selected z0: -10, where F0 = nan
Updated z0: -5.0, where F0 = nan
Updated z0: -2.5, where F0 = -313618.0887324887
Scipy will use -1.5 as z0 initial guess
Transfer trajectory is a hyperbolla; z = -1.513182442663566
Lagrange functions f: 0.3981234817955711, g: 1621.5355428908742, gdot: 0.5820811381357782
=========================== SOLUTION ===========================
Velocity at r1: [-10.23142384  -0.91347663   3.80129854] [km/s]
velocity at r2: [-8.32444572 -5.2695713   1.21771488] [km/s]


C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in power
C:\Users\yurio\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in sqrt


In [8]:
# solve Lambert problem in retrograde with more extreme tof
dt3 = 2
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt3, mu=mu_E, grade='retro')

========================= LAMBERT'S PROBLEM =========================
Transfering from r1: [ 5000 10000  2100] [km] 
              to r2: [-14600   2500   7000] [km] 
in time of flight: 2.3148148148148147e-05 [days]
dtheta: 259.70747579270375
Value of A: -12372.272033956451
Initially pre-selected z0: -10, where F0 = 1360160.1223655061
Scipy will use -10 as z0 initial guess
Transfer trajectory is a hyperbolla; z = -1017.6624120689979
Lagrange functions f: -6503183.549540431, g: -5330105.582516561, gdot: -4515548.956235075
=========================== SOLUTION ===========================
Velocity at r1: [ -6100.4238367  -12200.85362075  -2562.18047515] [km/s]
velocity at r2: [-12368.79996848   2117.94723685   5930.24740398] [km/s]
